# SMCC - Jupyter Example

In this example we will configure a stepper motor (3.75°/step) to run at a speed of 10 rpm and move a full rotation clockwise and counterclockwise. 

You only need the following imports to use the `SMCC`:

In [1]:
import serial
import time
from SMCC.Commands import SETUP_CMD, STEP_CMD, generateControlDict, checkConnection,getFrequency, sendCommand

# The following imports are not strictly necessary but they make the example look better
import pprint

## 1. Generating the base controls

The base of the `SMCC` is a dictionary that holds all the basic control implemented on the driver. To generate the controls you can use the `generateControlDict` function:

In [2]:
ctrl_dict = generateControlDict()

# Let's print the dict to show all the controls
pprint.pprint(ctrl_dict)

{'comms': None,
 'degrees_per_step': None,
 'direction': False,
 'enable': False,
 'freq': 0,
 'freq_counter': 0,
 'halt': False,
 'micro_stepping': <DEFAULTS.MICRO_STEPPING_DEFAULT: ['Full-Step', bitarray('00000000'), 1]>,
 'reset': True,
 'sleep': True,
 'speed': 1,
 'steps': 0}


The control dictionary holds:

1. The serial port (`comms`): This is the link between the `SMCC` and the Arduino board. By the default it's initialized with `None`. 

2. The base characteristic of the motor to control, the number of `degrees that the motor turns by a single step`. In this example we will use a motor with a relationship of: $$3.75~°/step$$

3. `direction`: Indicates if the motor should turn clockwise (True) or counterclockwise (False).

4. `enable`: This indicates if the Pololu Driver board should be active, works with negative logic, by default is always initialized to False (this means the Pololu board is enabled and waiting for commands). 

5. `freq`: Is the factible frequency that the controller should use to generate the step square wave. By default it's initialized to zero. 

6. `freq_counter`: Holds the value used by the Arduino to generate the requiered frequency square wave. By default it's initialized to zero.

7. `halt`: Indicates if the controller should stop all motor related operations. If the motor is performing a number of steps and the halt flag is raised and sent, the controller would stop and clean. 

8. `micro_stepping`: Indicates if the controller should use microstepping. By default the motor always is initialized with `FULL-STEP`. 

9. `reset`: Indicates if the Pololu board should be reseted, works with negative logic. If `False` the board resets until the value is `True`. 

10. `sleep`: Indicates if the Pololu board should sleep between step-operations. Works with negative logic, so when `False` the board will be slept in between operations. **This disables holding torque**.

12. `speed`: Holds the desired speed value in rpm. By default it's set to 1 rpm. 

11. `steps`: Indicates the number of steps that the motor should take. 


Now that we have analyzed every parameter available in the control dict, it's time to configure the controls for our particular case. 

## 2. Configuring the controls

First, we should inform the characteristics of our motor, this is used for the frequency /frequency counter calculations performed later:

In [3]:
ctrl_dict['degrees_per_step'] = 3.75

We need to set the direction of the spin, for clickwise we need to set direction to True:

In [4]:
ctrl_dict['direction'] = True

Then we should configure our desired speed and by extension the frequency of the square wave used to control each step of the motor:

In [5]:
ctrl_dict['speed'] = 10 # rpm]

To generate the corresponding frequency and counter, we can use the implemented function `getFrequency` that returns the ideal frequency, the factible frequency (the one that the Arduino controller can manage to create) and the associated counter for this factible frequency:

In [6]:
req_freq, real_freq, counter = getFrequency(ctrl_dict)
print('The requiered frequency is {} Hz\nThe factible frequency is {} Hz\nAnd the counter is {}'.format(req_freq, real_freq, counter))

The requiered frequency is 16.0 Hz
The factible frequency is 15.998976065531807 Hz
And the counter is 7812


We assign this valules to the control dictionary:

In [7]:
ctrl_dict['freq'] = real_freq
ctrl_dict['freq_counter'] = counter

After configuring the controls we need a way of telling the controller we want this settings to be applied, so we need to setup the serial communication!

## 3. Configuring serial port

For this we use `PySerial`. In the case of our example, the Arduino is connected to our computer via the `/dev/ttyACM0` serial port. You can use the Arduino-IDE interface to identify to which port your Arduino is connected to: 

<p align=center>
    <img src='images/example_serial_port.gif'>
</p>

We setup the connection as follows:

In [8]:
port = '/dev/ttyACM0' # Change this to the serial port your arduino is connected to
baud_rate = 9600 
timeout = 10.0 # seconds

ctrl_dict['comms'] = serial.Serial(port, baudrate=baud_rate, timeout=timeout)
# This allows the Arduino to reboot
time.sleep(10)

We setup the port with a timeout of 10 seconds, in case the Arduino doesn't reply to a command in that interval the connections stops and doesn't freeze. After setting up the connection we wait for 10 seconds for the Arduino to reboot, before this time there is no guarantee that the controller can reply to any sent command. 

We can test if the selected port can talk to the Arduino using the `checkConnection` function. The function sends a SETUP_CMD to the Arduino and check the reply to see if it matches the CMD:


In [9]:
if(checkConnection(ctrl_dict)):
    print('The connection is sucessful. Arduino reply matches sent CMD')
else:
    print('Something went wrong!')

The connection is sucessful. Arduino reply matches sent CMD


This process of comparing sent CMD and received response implemented in the checkConnection is also implemented on the `sendCommand` function. We will see some examples of this in the following section!

## 4. Sending the configurations to the Arduino

Now that we have customized the control dictionary, we need to send it to the Arduino Controller, for this we can use the function `SETUP_CMD` that generates a command that we can send to the controller using the function `sendCommand`: 

In [10]:
# Generate the setup cmd
setup_cmd = SETUP_CMD(ctrl_dict)

# Send Command
setup_response = sendCommand(ctrl_dict['comms'], setup_cmd)


`sendCommand` uses the port `ctrl_dict['comms']` to send the requested command to the Arduino. It also performs a check to verify that the controller reply matches the sent command, if it doesn't match raises an Exception. 


At this point, we have replicated the `ctrl_dict` configurations on the Arduino. Now we can proceed to send some step commands. 

## 5. Sending step command : Clockwise 360° rotation

We have setup the controls for this clockwise rotation in the previous sections. Now we have to generate the step command for the number of steps we want. Considering that out motor makes 3.75°/step we can calculate a full rotation as:

In [11]:
full_rotation_steps = int(360/ctrl_dict['degrees_per_step'])
print('Requiered steps for full rotation: {}'.format(full_rotation_steps))

Requiered steps for full rotation: 96


Now we set the step count on the `ctrl_dict` and prepare the step cmd using the `STEP_CMD` function:

In [12]:
ctrl_dict['steps'] = full_rotation_steps

# Prepare the command
step_cmd = STEP_CMD(ctrl_dict)

# Show the generated command
print('Show the generated step command:')
pprint.pprint(step_cmd)

Show the generated step command:
{'cmd': [bitarray('00000001'), bitarray('00000000'), bitarray('01100000')],
 'response_size': 3}


The command is sent using the `sendCommand` function:

In [13]:
step_response = sendCommand(ctrl_dict['comms'], step_cmd)

At this point, the motor shoul have performed a full clockwise rotation! Congratulations, now we can change the direction and try again. 

## 6. Sending Step Command: Counterclockwise 360° rotation

For this operation, we need to set the direction to counterclockwise (False), inform the arduino about this change and send a new step command:

In [14]:
# Change direction
ctrl_dict['direction'] = False

# Inform the arduino about this change
setup_cmd = SETUP_CMD(ctrl_dict)

# Send the changes to the Arduino
setup_response = sendCommand(ctrl_dict['comms'], setup_cmd)

# Generate a new step cmd
step_cmd = STEP_CMD(ctrl_dict)
# Inform the Arduino
step_response = sendCommand(ctrl_dict['comms'], step_cmd)


This concludes the tutorial, at this point the motor should have rotated one full rotation counterclockwise. Don't forget to run the following line to close the serial port: 

In [15]:
ctrl_dict['comms'].close()